In [2]:
"""
Stage 04 · Ratio Diagnostics & Leaderboards
===========================================

– Correlation (|ρ|) and Logit pseudo-R²/AUROC ranking for every ratio created
  in Stage 03, plus multicollinearity filtering and descriptive stats.

Dynamic file-finding logic
--------------------------
The script no longer assumes a *numeric* RUN_DATE folder.  If Stage 03 was run
earlier in the same session we reuse the in-memory `data_stage_3` frame.
Otherwise we locate the most-recent Stage 03 CSV by scanning every sub-folder
under  rff/outputs_rff/event=<SWAN_YEAR>/.

Outputs   (written to the same run folder discovered above)
  stage04/
      Stage4_raw_RatioDiagnostics.csv
      Stage4_winsor_RatioDiagnostics.csv
      Stage4_DroppedCollinearWinsor.csv
      Stage4_<raw|winsor>_RatioRanking.csv
      Stage4_<raw|winsor>_{Bucket|Stage|Domain|Overall}Top3.csv
      stage04.log
"""
from __future__ import annotations

# ── imports ──────────────────────────────────────────────────────────
from pathlib import Path
import os, sys, logging, yaml, warnings, io, re
from typing import Dict, List, Tuple, Set
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from numpy.linalg import LinAlgError
from statsmodels.tools.sm_exceptions import MissingDataError

# ═════════════════════════════════════════════════════════════════════
# 0 · PIPELINE CONFIG
# ═════════════════════════════════════════════════════════════════════
CFG_FILE = Path(os.getenv("PIPELINE_CFG", "pipeline_config.yaml")).expanduser()
if not CFG_FILE.is_file():
    raise FileNotFoundError(f"pipeline_config.yaml not found at {CFG_FILE}")

with CFG_FILE.open(encoding="utf-8") as fh:
    CFG: Dict = yaml.safe_load(fh) or {}

DEFAULTS   = CFG.get("defaults", {})
EVENTS     = {str(k): v for k, v in CFG.get("events", {}).items()}
ST4_OVR    = CFG.get("stage4", {})           # optional per-stage overrides

# ═════════════════════════════════════════════════════════════════════
# 1 · PARAMETERS
# ═════════════════════════════════════════════════════════════════════
SWAN_YEAR = int(os.getenv("SWAN_YEAR", next(iter(EVENTS))))
if str(SWAN_YEAR) not in EVENTS:
    raise KeyError(f"SWAN_YEAR={SWAN_YEAR} missing in pipeline_config events:")

DATE_COL      = ST4_OVR.get("date_col", "ReportDate")
ID_COL        = ST4_OVR.get("id_col",   "Symbol")
MC_THRESH     = float(os.getenv("MC_THRESH",    ST4_OVR.get("mc_thresh",    0.95)))
MIN_COVERAGE  = float(os.getenv("MIN_COVERAGE", ST4_OVR.get("min_coverage", 40)))

# ═════════════════════════════════════════════════════════════════════
# 2 · PATH RESOLUTION (robust, no numeric run-folder assumption)
# ═════════════════════════════════════════════════════════════════════
OUTPUT_ROOT = Path(DEFAULTS["OUTPUT_ROOT"]).expanduser()
EVENT_DIR   = OUTPUT_ROOT / f"event={SWAN_YEAR}"

# (a) user can override the run folder directly
RUN_DIR: Path | None = None
if os.getenv("RUN_DIR"):
    RUN_DIR = Path(os.getenv("RUN_DIR")).expanduser()
elif os.getenv("RUN_DATE"):
    RUN_DIR = EVENT_DIR / os.getenv("RUN_DATE")

# (b) auto-discover latest Stage-3 CSV if not provided
if RUN_DIR is None:
    if not EVENT_DIR.is_dir():
        raise FileNotFoundError(f"No outputs for event={SWAN_YEAR} at {EVENT_DIR}")
    # locate *any* Stage3 CSV inside event folder
    candidates = list(EVENT_DIR.glob("*/stage03/Stage3_Data_WithRatios.csv"))
    if not candidates:
        raise FileNotFoundError(
            f"No Stage3_Data_WithRatios.csv found under {EVENT_DIR}.  Run Stage 03."
        )
    # pick the newest file by mtime; could also sort by folder name
    STAGE3_FILE = max(candidates, key=lambda p: p.stat().st_mtime)
    RUN_DIR     = STAGE3_FILE.parents[1]          # …/<run_tag>/
else:
    STAGE3_FILE = RUN_DIR / "stage03" / "Stage3_Data_WithRatios.csv"
    if not STAGE3_FILE.is_file():
        raise FileNotFoundError(f"Stage-03 CSV not found at {STAGE3_FILE}")

RUN_DATE = RUN_DIR.name
OUT_DIR  = RUN_DIR / "stage04"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ═════════════════════════════════════════════════════════════════════
# 3 · LOGGER
# ═════════════════════════════════════════════════════════════════════
logging.basicConfig(
    level   = logging.INFO,
    format  = "%(asctime)s | %(levelname)-7s | %(message)s",
    handlers=[
        logging.FileHandler(OUT_DIR / "stage04.log", mode="w", encoding="utf-8"),
        logging.StreamHandler(sys.stdout),
    ],
)
logger = logging.getLogger(__name__)
logger.info("==========  STAGE 04: DIAGNOSTICS & LEADERBOARDS ==========")
logger.info("Config        : %s", CFG_FILE)
logger.info("SWAN_YEAR=%s  RUN_DATE=%s  RUN_DIR=%s", SWAN_YEAR, RUN_DATE, RUN_DIR)
logger.info("MC_THRESH=%.2f  MIN_COVERAGE=%.1f%%", MC_THRESH, MIN_COVERAGE)

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ═════════════════════════════════════════════════════════════════════
# 4 · LOAD DATA  (memory → disk fallback)
# ═════════════════════════════════════════════════════════════════════
if "data_stage_3" in globals():
    df_full = globals()["data_stage_3"].copy()
    logger.info("Stage 03 data reused from memory: %d rows", len(df_full))
else:
    df_full = pd.read_csv(STAGE3_FILE, parse_dates=[DATE_COL], low_memory=False)
    logger.info("Stage 03 CSV loaded: %d rows", len(df_full))

# keep observations **before** the crisis
pre_df = df_full[df_full[DATE_COL].dt.year < SWAN_YEAR].copy()
if pre_df.empty:
    raise ValueError(f"No observations before SWAN_YEAR={SWAN_YEAR}")
logger.info("Pre-SWAN sample size: %d rows", len(pre_df))

# ░░░░░░░░░░░░░░░░░░  RATIO COLUMNS  ░░░░░░░░░░░░░░░░░░
ratio_names: List[str] = sorted({c[:-4] for c in pre_df.columns
                                 if c.endswith("_raw") and c[:-4] in pre_df.columns})
raw_cols  = [f"{r}_raw" for r in ratio_names]
win_cols  = ratio_names    # winsorised copies exist from Stage 03
logger.info("Detected %d ratios (%d raw + %d winsor)",
            len(ratio_names), len(raw_cols), len(win_cols))

# ═════════════════════════════════════════════════════════════════════
# 5 · WINSORISATION & MULTICOLLINEARITY
# ═════════════════════════════════════════════════════════════════════
def winsorise(s: pd.Series, pct: float = 0.01) -> pd.Series:
    if s.notna().sum() < 3:
        return s
    lo, hi = np.nanpercentile(s.dropna(), [pct*100, (1-pct)*100])
    return s if lo == hi else s.clip(lo, hi)

wins_df = pre_df.copy()
for c in win_cols:
    wins_df[c] = winsorise(wins_df[c])

# Spearman |ρ| ≥ MC_THRESH → drop lower-coverage column
corr = wins_df[win_cols].corr(method="spearman").abs()
mask = np.triu(np.ones(corr.shape), 1).astype(bool)
high_pairs = corr.where(mask).stack().loc[lambda s: s >= MC_THRESH]

drop_mc: Set[str] = set()
for (c1, c2), _ in high_pairs.sort_values(ascending=False).items():
    if c1 in drop_mc or c2 in drop_mc:
        continue
    keep, drop = (c1, c2) if wins_df[c1].notna().mean() >= wins_df[c2].notna().mean() else (c2, c1)
    drop_mc.add(drop)

if drop_mc:
    logger.info("MC filter: dropping %d winsor columns", len(drop_mc))
    pd.Series(sorted(drop_mc), name="Dropped_Winsor_Columns")\
        .to_csv(OUT_DIR / "Stage4_DroppedCollinearWinsor.csv", index=False)
    win_cols = [c for c in win_cols if c not in drop_mc]
    wins_df.drop(columns=list(drop_mc), inplace=True)
else:
    logger.info("MC filter: no columns dropped")

# ═════════════════════════════════════════════════════════════════════
# 6 · DIAGNOSTIC STATS
# ═════════════════════════════════════════════════════════════════════
def _write_diag(base: pd.DataFrame, cols: List[str], tag: str):
    diag = (
        base.assign(Year=base[DATE_COL].dt.year)
            .groupby("Year")[cols]
            .agg(['mean', 'std', 'median', 'min', 'max', 'count'])
            .stack(level=1)
            .reset_index()
    )
    diag.to_csv(OUT_DIR / f"Stage4_{tag}_RatioDiagnostics.csv", index=False)

_write_diag(pre_df,  raw_cols,  "raw")
_write_diag(wins_df, win_cols, "winsor")
logger.info("Diagnostic CSVs written")

# ═════════════════════════════════════════════════════════════════════
# 7 · RANKING HELPERS
# ═════════════════════════════════════════════════════════════════════
METRICS    = ["NetIncome","EarningBeforeInterestAndTax","OperatingIncome",
              "EBITDA","OperatingCashFlow","FreeCashFlow","Cash",
              "CashAndCashEquivalents","TotalRevenue","GrossProfit"]
score_cols = [f"Score_{m}" for m in METRICS]
flag_cols  = [f"Flag_{m}"  for m in METRICS]

def _abs_spearman(x: pd.Series, y: pd.Series, min_obs=100):
    ok = x.notna() & y.notna()
    return np.nan if ok.sum() < min_obs else abs(spearmanr(x[ok], y[ok]).correlation)

def _logit_stats(x: pd.Series, y_flag: pd.Series, min_obs=100) -> Tuple[float,float]:
    ok = x.notna() & y_flag.isin([0,1])
    if ok.sum() < min_obs or y_flag[ok].nunique() < 2 or x[ok].nunique() < 2:
        return np.nan, np.nan
    try:
        mdl = sm.Logit(y_flag[ok], sm.add_constant(x[ok])).fit(disp=False)
        return mdl.prsquared, roc_auc_score(y_flag[ok], mdl.predict())
    except (ValueError, LinAlgError, MissingDataError):
        return np.nan, np.nan

# bucket / stage / domain maps (if Stage 03 executed this session)
ratio_to_buckets: Dict[str, List[str]] = {
    r: (bs if isinstance(bs, (list, tuple)) else [bs])
    for r, bs in globals().get("ratio_domain_stage_map", {}).items()
}
domains = {"Phys", "Info", "Cog", "Soc"}
stages  = {"Prepare", "Absorb", "Recover", "Adapt"}

# ═════════════════════════════════════════════════════════════════════
# 8 · RANK LOOP
# ═════════════════════════════════════════════════════════════════════
def _rank(tag: str, cols: List[str], base: pd.DataFrame):
    logger.info("Ranking %s (%d columns)…", tag, len(cols))

    rows = []
    for col in cols:
        cov_pct = base[col].notna().mean()*100
        if cov_pct < MIN_COVERAGE:
            continue
        x = base[col]
        for m, sc, fc in zip(METRICS, score_cols, flag_cols):
            rho  = _abs_spearman(x, base[sc])
            pr2, auc = _logit_stats(x, base[fc])
            rows.append({"Ratio": col.replace("_raw",""),
                         "Metric": m,
                         "Coverage%": round(cov_pct, 1),
                         "|rho|": round(rho, 3) if pd.notna(rho) else np.nan,
                         "PseudoR2": round(pr2, 3) if pd.notna(pr2) else np.nan,
                         "AUROC":   round(auc, 3) if pd.notna(auc) else np.nan})
    rk = pd.DataFrame(rows)
    rk.to_csv(OUT_DIR / f"Stage4_{tag}_RatioRanking.csv", index=False)

    # ---- selector maps ------------------------------------------------------
    bucket_map = ratio_to_buckets or {"All": [c.replace("_raw","") for c in cols]}
    stage_map  = {
        stg: [r for r, bs in bucket_map.items()
              if any(str(b).endswith(f"-{stg}") for b in bs)]
        for stg in stages
    }
    domain_map = {
        dom: [r for r, bs in bucket_map.items()
              if any(str(b).startswith(dom) for b in bs)]
        for dom in domains
    }

    def _top3(df):  # pick top-3 by |rho|
        return df.nlargest(3, "|rho|")

    # (a) bucket leaderboard
    bucket_rows = []
    for bucket, members in bucket_map.items():
        sub = rk[rk["Ratio"].isin(members)]
        for metric in METRICS:
            for _, r in _top3(sub[sub["Metric"] == metric]).iterrows():
                bucket_rows.append({"Bucket": bucket, **r.drop("Ratio")})
    pd.DataFrame(bucket_rows).to_csv(OUT_DIR / f"Stage4_{tag}_BucketTop3.csv",
                                     index=False)

    # (b) stage leaderboard
    stage_rows = []
    for stg, members in stage_map.items():
        sub = rk[rk["Ratio"].isin(members)]
        for metric in METRICS:
            for _, r in _top3(sub[sub["Metric"] == metric]).iterrows():
                stage_rows.append({"Stage": stg, **r.drop("Ratio")})
    pd.DataFrame(stage_rows).to_csv(OUT_DIR / f"Stage4_{tag}_StageTop3.csv",
                                    index=False)

    # (c) domain leaderboard
    domain_rows = []
    for dom, members in domain_map.items():
        sub = rk[rk["Ratio"].isin(members)]
        for metric in METRICS:
            for _, r in _top3(sub[sub["Metric"] == metric]).iterrows():
                domain_rows.append({"Domain": dom, **r.drop("Ratio")})
    pd.DataFrame(domain_rows).to_csv(OUT_DIR / f"Stage4_{tag}_DomainTop3.csv",
                                     index=False)

    # (d) overall leaderboard
    overall = []
    for metric in METRICS:
        for _, r in _top3(rk[rk["Metric"] == metric]).iterrows():
            overall.append(r.drop("Ratio").to_dict())
    pd.DataFrame(overall).to_csv(OUT_DIR / f"Stage4_{tag}_OverallTop3.csv",
                                 index=False)

# run rankings
_rank("raw",    raw_cols, pre_df)
_rank("winsor", win_cols, wins_df)

logger.info("✅  STAGE 04 complete — artefacts in %s", OUT_DIR)

2025-06-10 12:53:15,321 | INFO    | ==========  STAGE 04: DIAGNOSTICS & LEADERBOARDS ==========
2025-06-10 12:53:15,324 | INFO    | Config        : pipeline_config.yaml
2025-06-10 12:53:15,326 | INFO    | SWAN_YEAR=2008  RUN_DATE=20250609  RUN_DIR=C:\Users\Jason Pohl\OneDrive - Bond University\PhD\rff\outputs_rff\event=2008\20250609
2025-06-10 12:53:15,327 | INFO    | MC_THRESH=0.95  MIN_COVERAGE=40.0%
2025-06-10 12:53:20,963 | INFO    | Stage 03 CSV loaded: 34862 rows
2025-06-10 12:53:21,058 | INFO    | Pre-SWAN sample size: 11204 rows
2025-06-10 12:53:21,060 | INFO    | Detected 229 ratios (229 raw + 229 winsor)


C:\Users\Jason Pohl\AppData\Local\Temp\ipykernel_35848\230431602.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return s if lo == hi else s.clip(lo, hi)


2025-06-10 12:53:37,202 | INFO    | MC filter: dropping 65 winsor columns
2025-06-10 12:53:38,402 | INFO    | Diagnostic CSVs written
2025-06-10 12:53:38,404 | INFO    | Ranking raw (229 columns)…


c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2025-06-10 12:54:30,954 | INFO    | Ranking winsor (164 columns)…


c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Jason Pohl\miniconda3\lib\site-packages\statsmodels\base\model.py:6

2025-06-10 12:55:17,290 | INFO    | ✅  STAGE 04 complete — artefacts in C:\Users\Jason Pohl\OneDrive - Bond University\PhD\rff\outputs_rff\event=2008\20250609\stage04
